# Ciencia De Datos Aplicada (ITBA): Segundo Entregable

## Recopilación y Preparación de Datos

**Equipo:** Uriel Arias - 63504, Cristian Tepedino - 62830

**Nombre del proyecto**: Sistema de predicción de resultados de
carreras de Fórmula 1

### 🧾 1. Importación y carga de librerías

In [3]:
!pip install fastf1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 5.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [21]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import fastf1


cache_dir = './cache'
os.makedirs(cache_dir, exist_ok=True)

fastf1.Cache.enable_cache(cache_dir)

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

import warnings
warnings.filterwarnings("ignore")

### 🧾 2. Carga de datos

In [ ]:
#Cargar sesión de ejemplo
session = fastf1.get_session(2021, 'Italian Grand Prix', 'Q')
session.load()

#Obtener laps y filtrar vueltas rápidas válidas
laps = session.laps
laps = laps.pick_quicklaps()

#Obtener telemetría por piloto
telemetry_features = []

for driver in session.drivers:
    driver_laps = laps.pick_driver(driver)
    driver_info = session.get_driver(driver)
    driver_name = f"{driver_info['FirstName']} {driver_info['LastName']}"

    for _, lap in driver_laps.iterlaps():
        lap_telemetry = lap.get_telemetry()
        telemetry_features.append({
            'Driver': driver,
            'DriverName': driver_name,
            'LapNumber': lap.LapNumber,
            'LapTime': lap.LapTime.total_seconds(),
            'Speed_mean': lap_telemetry['Speed'].mean(),
            'Throttle_mean': lap_telemetry['Throttle'].mean(),
            'Brake_mean': lap_telemetry['Brake'].mean(),
            'RPM_mean': lap_telemetry['RPM'].mean(),
            'DRS_max': lap_telemetry['DRS'].max()
        })

df_telemetry = pd.DataFrame(telemetry_features)

#Agregar datos de clima
weather = session.weather_data
weather_summary = weather.mean().to_dict()

# Agregamos clima a cada fila del DataFrame
for key, value in weather_summary.items():
    df_telemetry[key] = value

#Revisar el DataFrame final
df_telemetry.head()

core           INFO 	Loading data for Italian Grand Prix - Qualifying [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['77', '44', '33', '4', '3', '10', '55', '16', '11', '99', '5', '18', '14', '31', '63', '6', '22', '47', '88', '9']


### 🗒️ 3. Descripción del dataset
Este dataset contiene información por vuelta de cada piloto durante una sesión de Fórmula 1, incluyendo tiempos de vuelta, telemetría y condiciones de clima.

**Variables principales:**

- Driver:	Código del piloto
- DriverName: Nombre del piloto
- LapNumber:	Número de vúelta
- LapTime:	Tiempo total de la vuelta (en segundos)
- Speed_mean:	Velocidad promedio durante la vuelta (km/h)
- Throttle_mean: Porcentaje promedio de acelerador durante la vuelta
- Brake_mean:	Porcentaje promedio de freno durante la vuelta
- RPM_mean:	Revoluciones promedio del motor durante la vuelta
- DRS_max:	Valor máximo del DRS en la vuelta (0 = cerrado, 1 = abierto)

**Variables de clima promedio durante la sesión:**

- Temperature:	Temperatura del aire (°C)
- TrackTemperature:	Temperatura de la pista (°C)
- Humidity:	Humedad relativa (%)
- Pressure:	Presión atmosférica (hPa)
- WindSpeed:	Velocidad del viento (km/h)
- WindDirection:	Dirección del viento (grados)

**Variable objetivo potencial:**

- LapTime: regresión para predecir el tiempo de vuelta de un piloto.
- Position: clasificación del ganador/top3/top10 por vuelta o por sesión

**Notas:**

- Cada fila corresponde a una vuelta de un piloto.
- La telemetría se ha agregado usando estadísticas por vuelta (media o máximo) para reducir el número de filas y simplificar el modelado.
- Las variables de clima son promedios de toda la sesión, y pueden servir como features adicionales para mejorar la predicción del desempeño.

### 🔍 4. Análisis exploratorio de datos (EDA)

### 🧼 5. Diagnóstico de calidad de datos


### 🧪 6. Transformaciones realizadas

### ✍️ 7. Reflexión final
